# <font color='#4C5FDA'>**Breast Cancer Detection Based on CNNs Using Thermal Imaging** </font>

Original paper by Juan Pablo Zuluaga, Zeina Al Masry, Khaled Benaggoune, Safa Meraghni & Noureddine Zerhouni: [A CNN-based methodology for breast cancer diagnosis using thermal images](https://www.tandfonline.com/doi/full/10.1080/21681163.2020.1824685)

In [ ]:
#@title **Instalar paquetes necesarios**

%%capture
! pip install torchmetrics
! pip install wandb -Uq
# ! pip install onnx

## <font color='#ECA702'>**Clonamos nuestro repo**</font>

Esto con el fin de traer todos los .py para poder entrenar 'localmente' en Colab y registrar las métricas en wandb.

In [ ]:
! git clone https://github.com/gpintoruiz/Thermal-Imaging-Breast-Cancer-Detection.git

Cloning into 'Thermal-Imaging-Breast-Cancer-Detection'...
remote: Enumerating objects: 418, done.
remote: Counting objects: 100% (198/198), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 418 (delta 125), reused 86 (delta 44), pack-reused 220
Receiving objects: 100% (418/418), 7.79 MiB | 13.03 MiB/s, done.
Resolving deltas: 100% (252/252), done.


In [ ]:
%cd Thermal-Imaging-Breast-Cancer-Detection/notebooks

/content/Thermal-Imaging-Breast-Cancer-Detection/notebooks


In [ ]:
%ls

0.01-gpr-data-exploration-cv2.ipynb   make_dataset.py  train_one_run.py       xception-one-run.yaml
0.01-gpr-data-exploration-pil.ipynb   preprocess.py    train.py               xception.py
0.04-gpr-colab-experiments.ipynb      resnet56.py      utils.py
1.00-gpr-xception-from-scratch.ipynb  test.py          vgg.py
alexnet.py                            train_gkfold.py  xception-gfkfold.yaml


## <font color='#ECA702'>**Configuración inicial para conectarnos con Kaggle**</font>

1. Instalamos kaggle. Para poder usar comandos de Kaggle.

In [ ]:
%%capture
! pip install kaggle

Subimos nuestro token de autenticación de Kaggle (si estamos en colab, sino colocarlo en la carpeta del proyecto)

In [ ]:
from google.colab import files
files.upload()

1. Creamos los directorios de Kaggle
2. Copiamos nuestro token en .kaggle
3. Con `chmod 600` establecemos los permitos del token en 600, es decir, que solo yo tengo permisos de lectura y escritura sobre el archivo

In [ ]:
! mkdir ~/kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

## <font color='#ECA702'>**Carga del dataset**</font>

Traemos el dataset [Thermal Images for Breast Cancer Diagnosis DMR-IR](https://www.kaggle.com/datasets/asdeepak/thermal-images-for-breast-cancer-diagnosis-dmrir) desde kaggle.

This dataset is a methodology for breast disease computer-aided diagnosis using dynamic thermography. The thermal images for breast tumors are classified according to DMR-IR standards.

Two types of tumors are classified in this dataset one is benign another is malignant.
- Benign: This type of tumor is usually well-defined and round or oval in shape. (non-cancerous tumor)
- Malignant: This type of tumor is usually poorly defined and irregular with lobules. (cancerous tumor)

In [ ]:
%%capture
! kaggle datasets download -d asdeepak/thermal-images-for-breast-cancer-diagnosis-dmrir
! unzip thermal-images-for-breast-cancer-diagnosis-dmrir.zip

Después de descargar los datos. Debemos entender la estructura de las carpetas para poder trabajar con ellas de una mejor manera.
1. La carpeta principal `Imagens e Matrizes da Tese de Thiago Alves Elias da Silva` son todos los datos `data`.
2. La carpeta `12 Novos Casos de Testes` la podemos tomar como nuestro conjunto de prueba (`test`).
3. Mientras que la carpeta `Desenvolvimento da Metodologia` será nuestro conjunto de entrenamiento (`train`).

Luego dentro de nuestras carpetas de `train` y `test` encontramos dos categorías `DOENTES`y `SAUDAтХа├╝VEIS` o SAUDÁVEI. Los primeros son los casos malignos y los segundos benignos.

Dentro de cada una de las carpetas de pacientes saludables y enfermos se encuentran carpetas con números, cada número representa un paciente. Y para cada paciente tendremos dos carpetas más, una para las imágenes **segmentadas** en escala de grises y la otra para la matrix o mapa de calor.

Algo bueno de este dataset es que ya está dividido por pacientes, es decir, no tendremos imagenes del mismo paciente en el conjunto de entrenamiento y testeo. Por lo tanto, vamos a entrenar con N pacientes, y testear con K pacientes, que no son los mismos.

### <font color='#52F17F'>**Partición de los datos**</font>

Este comando nos permite cargar funciones de un .py en el entorno local de Colab. [Fuente](https://stackoverflow.com/questions/47345004/in-googles-colab-notebook-how-do-i-call-a-function-from-a-python-file)

In [ ]:
execfile('make_dataset.py')

In [ ]:
np.random.seed(2024)

def print_fold_patients(folds: dict, data: pd.DataFrame):
    for fold_name, indices in folds.items():
        train_patients = data.iloc[indices['train']]['patient'].unique()
        # val_patients = data.iloc[indices['val']]['patient'].unique()
        test_patients = data.iloc[indices['test']]['patient'].unique()

        print(f"{fold_name}:\n")
        print(f"Train patients: {train_patients}")
        # print(f"Validation patients: {val_patients}")
        print(f"Test patients: {test_patients}\n")

# Generar los datos
data = make_dataframe()

# Generar los folds
folds = make_folds(data)

# Imprimir los pacientes por cada fold
print_fold_patients(folds, data)

fold_1:

Train patients: ['48' '14' '69' '63' '66' '65' '62' '46' '15' '45' '43' '13' '38' '44'
 '39' '41' '42' '16' '40' '37' '17' '36' '05' '53' '31' '03' '06' '55'
 '56' '24' '01' '51' '09' '04' '00' '25' '58' '52' '02' '49' '29' '35'
 '27' '32' '08' '59' '30' '28']
Test patients: ['61' '64' '34' '11' '26' '07' '60' '54']

fold_2:

Train patients: ['48' '14' '63' '61' '66' '64' '65' '15' '45' '43' '13' '38' '44' '39'
 '41' '42' '16' '34' '11' '40' '37' '17' '36' '03' '06' '55' '56' '24'
 '26' '01' '51' '09' '04' '00' '25' '58' '52' '07' '60' '49' '29' '35'
 '32' '08' '59' '54' '30' '28']
Test patients: ['69' '62' '46' '05' '53' '31' '02' '27']

fold_3:

Train patients: ['48' '69' '61' '66' '64' '65' '62' '46' '15' '43' '13' '38' '44' '39'
 '41' '42' '16' '34' '11' '40' '37' '05' '53' '31' '03' '55' '56' '24'
 '26' '01' '51' '09' '04' '00' '25' '58' '07' '60' '02' '49' '29' '35'
 '27' '32' '08' '59' '54' '30']
Test patients: ['14' '63' '45' '17' '36' '06' '52' '28']

fold_4:

Train p

## <font color='#ECA702'>**Inicializamos el agende de wandb**</font>

### <font color='#52F17F'>**1. Nos logeamos en nuestra cuenta**</font>

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### <font color='#52F17F'>**2. Hacemos call del agente**</font>


El sweep que estoy probando acá es el [siguiente](https://github.com/gpintoruiz/Thermal-Imaging-Breast-Cancer-Detection/blob/main/notebooks/xception-one-run.yaml). Se pueden cambiar los parámetros a probar como tú quieras de acuerdo con la [documentación](https://docs.wandb.ai/guides/sweeps/define-sweep-configuration) (recomiendo solo cambiar la arquitectura para que las comparaciones entre modelos sean equivalentes).

Si no tienes ni idea qué es un sweep mira el siguiente [tutorial](https://www.youtube.com/watch?v=9zrmUIlScdY&t=1361s&ab_channel=Weights%26Biases).

El comando `--count` sirve para decirle al agente cuántos runs hacer, aplica especialmente cuando el método del sweep es `bayes` o `random`


In [ ]:
! wandb agent aiuis/dip-project/5oq5gka9

wandb: Starting wandb agent 🕵️
2024-07-04 18:22:41,524 - wandb.wandb_agent - INFO - Running runs: []
2024-07-04 18:22:41,716 - wandb.wandb_agent - INFO - Agent received command: run
2024-07-04 18:22:41,716 - wandb.wandb_agent - INFO - Agent starting run with config:
	architecture: xception
	augmented: True
	batch_size: 64
	crop: True
	learning_rate: 0.0006177338121893195
	normalize: False
	optimizer: adam
2024-07-04 18:22:41,718 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python train_gkfold.py --architecture=xception --augmented=True --batch_size=64 --crop=True --learning_rate=0.0006177338121893195 --normalize=False --optimizer=adam
2024-07-04 18:22:46,730 - wandb.wandb_agent - INFO - Running runs: ['51i3330j']
wandb: Currently logged in as: gpintoruiz (aiuis). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: Tracking run with wandb version 0.17.4
wandb: Run data is saved locally in /content/T